In [1]:
from imutils import paths
from tqdm import tqdm
import pandas as pd 
import numpy as np
import shutil
import cv2
import os


TRAIN_DATASET_PATH = "ucfTrainTestlist/trainlist03.txt"
TEST_DATASET_PATH = "ucfTrainTestlist/testlist03.txt"

In [2]:
# Open the .txt file which have names of training videos
f = open(TRAIN_DATASET_PATH, "r")
temp = f.read()
videos = temp.split('\n')

# Create a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c01.avi 1
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi 1
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c03.avi 1
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c04.avi 1
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c05.avi 1


In [3]:
# Open the .txt file which have names of test videos
with open(TEST_DATASET_PATH, "r") as f:
    temp = f.read()
videos = temp.split("\n")

# Create a dataframe having video names
test = pd.DataFrame()
test["video_name"] = videos
test = test[:-1]
test.head()

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c01.avi
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c02.avi
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c03.avi
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c04.avi
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g15_c05.avi


In [4]:
def extract_tag(video_path):
    return video_path.split("/")[0]

def separate_video_name(video_name):
    return video_name.split("/")[1]

def rectify_video_name(video_name):
    return video_name.split(" ")[0]

def move_videos(df, output_dir):
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    for i in tqdm(range(df.shape[0])):
        videoFile = df['video_name'][i].split("/")[-1]
        videoPath = os.path.join("data", videoFile)
        shutil.copy2(videoPath, output_dir)
    print()
    print(f"Total videos: {len(os.listdir(output_dir))}")

In [5]:
train["tag"] = train["video_name"].apply(extract_tag)
train["video_name"] = train["video_name"].apply(separate_video_name)
train.head()

,video_name,tag
0,v_ApplyEyeMakeup_g01_c01.avi 1,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g01_c02.avi 1,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g01_c03.avi 1,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g01_c04.avi 1,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g01_c05.avi 1,ApplyEyeMakeup


In [6]:
train["video_name"] = train["video_name"].apply(rectify_video_name)
train.head()

,video_name,tag
0,v_ApplyEyeMakeup_g01_c01.avi,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g01_c02.avi,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g01_c03.avi,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g01_c04.avi,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g01_c05.avi,ApplyEyeMakeup


In [7]:
test["tag"] = test["video_name"].apply(extract_tag)
test["video_name"] = test["video_name"].apply(separate_video_name)
test.head()

,video_name,tag
0,v_ApplyEyeMakeup_g15_c01.avi,ApplyEyeMakeup
1,v_ApplyEyeMakeup_g15_c02.avi,ApplyEyeMakeup
2,v_ApplyEyeMakeup_g15_c03.avi,ApplyEyeMakeup
3,v_ApplyEyeMakeup_g15_c04.avi,ApplyEyeMakeup
4,v_ApplyEyeMakeup_g15_c05.avi,ApplyEyeMakeup


In [8]:
n = 2
topNActs = train["tag"].value_counts().nlargest(n).reset_index()["index"].tolist()
train_new = train[train["tag"].isin(topNActs)]
test_new = test[test["tag"].isin(topNActs)]
train_new.shape, test_new.shape

((247, 2), (84, 2))

In [9]:
train_new = train_new.reset_index(drop=True)
test_new = test_new.reset_index(drop=True)

In [10]:
train_new.to_csv("test_train03.csv", index=False)
test_new.to_csv("test_test03.csv", index=False)